# TrieSort

 Sorting algorithm based on trie data structure

> # Introduction
 We have comparison sorts like merge sort, insert sort, quick sort etc with best runtime complexity of O(nlogn) 
 and we have non comparison sorts like radix, bucket sort and counting sort but with limitations.
 Today we are going to introduce TrieSort using the Trie data structure and using n comparisons. It's runtime 
 complexity is O(nlogm) where m is the len(str(max(nums))) where nums is the input array. The log part doesn't depend
 on the number of items in the list and it's to the base 10. It's not a comparison sort and uses index to avoid
 comparisons. Let's begin

> # The logic <br>
 Let's start from the simplest base case <br>
 Assume you have a dict (Trie is implemented using a recursive dict) and a list nums whose range is between 0 and 9
 Now if you keep inserting those elements in the dict with keys as the number itself you will have a dict like this

In [37]:
from random import randrange
import collections
# the dictionary we are gonna use to sort. 
sort_dict = collections.defaultdict(list)
for i in range(0, 9):
    el = randrange(10)
    sort_dict[el] = True # Value doesn't matter. Yet

In [23]:
sort_dict # we can see there were some duplicate numbers also. Don't worry we will take care of it later

defaultdict(list, {1: True, 8: True, 9: True, 0: True, 7: True, 2: True})

>   ## Pay attention. 
This is the trick. We know the dict have numbers from zero to nine. If we try every key from zero to nine some might be present others might not. We put everything that is present in a list in that order then we technically have our sorted list. Without any comparisons at all. 

Don't get too excited, we will need comparisons later.

In [24]:
sorted_list = [x for x in range(10) if sort_dict.get(x)]
sorted_list

[0, 1, 2, 7, 8, 9]

> # TADA!!!

Is it good? Someone just have one element to sort or two. Triesort will still create a dict and loop through zero to ninne to find out which digit that is. It might look like for small data set the comparison sorts are better. Too bad.

And also we sorted numbers who have only one digit each. Some guy might ask 

>  What if we had list of numbers upto 100 to sort?? 

 #### Opinion. 
 'Guy' should be gender neutral.

 #### Popular opinion.
 That guy could have asked for numbers upto 1000 to sort or a thousand million (billion!?). But let's take 100 for sake of simplicity.


 We can use a dict with 100 keys but where is the fun in that? Also it's not very efficient if you get two numbers to sort and one is one and other is one million. You would need to loop over a million keys to sort it. Doesn't sound TADA.

> In comes the <b>Trie Data Structure.</b> Mostly used in autocompletion or for checking prefix.

# Pay some more attention
With Trie you can build the tree with each digit of the numbers in the list. If we have a 5 in the list and the max value in the list is 99 then we would need two layer of the trie nodes. And 5 won't be just 5, it would be 05. Makes a big difference.

In [1]:
import collections
class TrieNode:
    def __init__(self):
        self.children = collections.defaultdict(TrieNode)
        
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        current = self.root
        for w in word:
            current = current.children[w]

That's our trie implementation, nothing special. Let's build the TrieSort class

In [4]:
class TrieSort:
    def __init__(self):
        self.res = []
        
    def buildTrie(self, nums):
        trie = Trie()
        max_num_len = len(str(max(nums))) # We get the highest number's no. of digits 
        for num in nums:
            num_len = len(str(num))
            zero_len = max_num_len - num_len # Substitute with zeros if it don't have the max len
            trie.insert('0'* zero_len + str(num))
        return trie.root

In [15]:
trie_sort = TrieSort()
trie = trie_sort.buildTrie([5, 55, 435, 666, 12321, 55, 5])

The biggest number in the list is 12321 which has 5 digits. So the element 5 or 55 or 435 all should have 5 elements
for this to work. So we put zeroes in front of them


In [16]:
trie.children

defaultdict(__main__.TrieNode,
            {'0': <__main__.TrieNode at 0x7fdb4f67cd00>,
             '1': <__main__.TrieNode at 0x7fdb4fda6f70>})

In [17]:
trie.children['1'].children 

defaultdict(__main__.TrieNode, {'2': <__main__.TrieNode at 0x7fdb4fda6d30>})

In [18]:
trie.children['0'].children 

defaultdict(__main__.TrieNode, {'0': <__main__.TrieNode at 0x7fdb4f682250>})

In [14]:
trie.children['0'].children['0'].children

defaultdict(__main__.TrieNode,
            {'0': <__main__.TrieNode at 0x7fdb4fd7ed30>,
             '4': <__main__.TrieNode at 0x7fdb4f3f6bb0>,
             '6': <__main__.TrieNode at 0x7fdb4f3f69d0>})

The '1' node at the immediate child of root have 2,3,2,1 in it's depth nodes in subsequent layers and '0' have '0' and you see the pattern right?
<br> That '4' in the third layer of '0' node belongs to the number '435' and '6' belongs to '666' and '0' belongs to '55' and also the '05' in there

> # The logic II - Sorting the list?<br>
From here it's easy but still pay attention, you do a dfs through this trie at each node and at each node you don't look for the keys, you know the keys will be in the range zero to nine at every node. Why not loop over it through that range? We loop in the order, obviously, and we ignore the values for which the nodes are missing. 

In [24]:
    def trieSort(self, node, word, depth):
        for i in range(0, 10):
            i = str(i)
            if not node.children.get(i):
                continue
            if depth == max_num_len-1: 
                self.res.append(int(word+i))
            self.joSort(node.children.get(i), word + i, depth +1)

 It's part of the TrieSort class and just updates the 'res' variable with the sorted results. 'res' is updated only when we are in the leaf node.

> # TADA!!!

That same guy comes around and asks about duplicates. If you have used trie before you would by now know what to do.

> # The logic III - Handle duplicates<br>
In our trie class we can add another variable called count, which as you have already guessed, counts the number of times that node has been visited. We just need to update the leaf nodes. Let's update the code

In [23]:
import collections
class TrieNode:
    def __init__(self):
        self.children = collections.defaultdict(TrieNode)
        # count variable
        self.count = 0
        
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        current = self.root
        for w in word:
            current = current.children[w]
        # update at the leaf node
        current.count += 1

Now the buildTrie method can remain the change but we need to update the trieSort method to accomodate duplicates

In [26]:
    def trieSort(self, node, word, depth):
        for i in range(0, 10):
            i = str(i)
            if not node.children.get(i):
                continue
            if depth == max_num_len-1:
                # append the number as much times as the count
                self.res += [int(word+i)] * node.children.get(i).count
            self.joSort(node.children.get(i), word + i, depth +1)

I believe that's another TADA moment

> # TADA!!!

# Now let's put it all together.


In [34]:
import collections
class TrieNode:
    def __init__(self):
        self.children = collections.defaultdict(TrieNode)
        self.count = 0
        
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        current = self.root
        for w in word:
            current = current.children[w]
        current.count += 1

class TrieSort:
    def __init__(self):
        self.res = []
        self.max_num_len = 0
        
    def buildTrie(self, nums):
        trie = Trie()
        self.max_num_len = len(str(max(nums))) # We get the highest number's max value of digits 
        for num in nums:
            num_len = len(str(num))
            zero_len = self.max_num_len - num_len
            trie.insert('0'* zero_len + str(num))
        return trie.root

    def trieSort(self, node, word, depth):
        for i in range(0, 10):
            i = str(i)
            if not node.children.get(i):
                continue
            if depth == self.max_num_len-1:
                self.res += [int(word+i)] * node.children.get(i).count
            self.trieSort(node.children.get(i), word + i, depth +1)
            
    def sort(self, nums):
        trie = self.buildTrie(nums)
        # we start at root node with an empty word and at depth zero
        self.trieSort(trie, "", 0)
        return self.res

In [35]:
TrieSort().sort([5, 55, 435, 666, 12321, 55, 5])

[5, 5, 55, 55, 435, 666, 12321]

Let's have a laugh and look into the time and space complexities

> # BU HA HA HA HA !

 # Time complexity
Building the Trie: For each node it takes log(len(str(max(nums))) to the base 10. For m digits (len(str(max(nums)))) in maximum value it would take O(nLogm) time to insert each value into the trie <br><br>
trieSort method is gonna take atmost O(nlog m) time. Duplicates makes this function more efficient. Also the for loop in trieSort runs exactly 10 times at a node. <br> <br>
Finding the max value in the array is gonna take O(n) time. <br> <br>
So max time complexity in worst case would be O(nlog m). 'm' is len(str(max(nums))) which doesn't depend on n, the number of elements, but the elements to be sorted



 # Space complexity
At worst case where we have all the numbers from 0 to 10^m - 1 is present we will need atmost O(nlog m) (to the base 10) space

The same gender neutral guy comes back and asks what about the negative numbers or the alphabhets and special characters or the decimals. We tell them the same thing we say to the god of death.

PS: It's easy. Need one layer at the top to assign the sign of the value. Two nodes, plus and minus. <br><br>
Alphabets and special characters can also be easily accomodated. Instead of range from zero to nine, we add them too. That will affect the time and space complexities<br><br>
Decimals are also easy. Just need to add a '.' node after the max_num_len depth node.

> # Conclusion
This is too good to be true. Lemme know how I'm making an arse of myself. <br><br> You can also convert them into binaries and the time and space complexities would still be O(nlogm) but log to the base 2. and m would be log(len(str(max(nums))) to the base 2. 2*50 or 10*10. Need to confirm if it makes any difference. Lemme know.